# 用于预训练BERT的数据集

为了预训练`sec_bert`中实现的BERT模型，我们需要以理想的格式生成数据集，以便于两个预训练任务：遮蔽语言模型和下一句预测。一方面，最初的BERT模型是在两个庞大的图书语料库和英语维基百科（参见`subsec_bert_pretraining_tasks`）的合集上预训练的，但它很难吸引这本书的大多数读者。另一方面，现成的预训练BERT模型可能不适合医学等特定领域的应用。因此，在定制的数据集上对BERT进行预训练变得越来越流行。为了方便BERT预训练的演示，我们使用了较小的语料库WikiText-2`Merity.Xiong.Bradbury.ea.2016`。

与`sec_word2vec_data`中用于预训练word2vec的PTB数据集相比，WikiText-2（1）保留了原来的标点符号，适合于下一句预测；（2）保留了原来的大小写和数字；（3）大了一倍以上。

In [21]:
import os
import random
import torch
from d2l import torch as d2l
import pandas as pd

在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。保留至少有两句话的段落。

为了简单起见，我们仅使用句号作为分隔符来拆分句子。我们将更复杂的句子拆分技术的讨论留在本节末尾的练习中。

In [12]:
'''
#@save
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]  # 确保文本行至少包含两个子句
    # 随机打乱 paragraphs 列表中段落的顺序
    random.shuffle(paragraphs)
    return paragraphs
'''

WikiTextDataset 链接已经失效，需要按照：[wikitext-2-v1 无法使用的问题](https://discuss.d2l.ai/t/wikitext-2-v1/22813/1) 对代码进行更改。

In [22]:
# wikitext-2-v1 无法使用的问题: https://discuss.d2l.ai/t/wikitext-2-v1/22813/1

def _read_wiki1(data_dir):
    # 读取 Parquet 文件为 DataFrame
    df = pd.read_parquet(data_dir)
    lines = df['text'].tolist()
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]  # 确保文本行至少包含两个子句
    random.shuffle(paragraphs)  #在段落间进行打乱
    return paragraphs

## 为预训练任务定义辅助函数

在下文中，我们首先为BERT的两个预训练任务实现辅助函数。这些辅助函数将在稍后将原始文本语料库转换为理想格式的数据集时调用，以预训练BERT。

### 生成下一句预测任务的数据

根据`subsec_nsp`的描述，`_get_next_sentence`函数生成二分类任务的训练样本。

In [13]:
#@save
def _get_next_sentence(sentence, next_sentence, paragraphs):
    # 以 50% 的概率决定是否随机选择下一句话
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs 是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

下面的函数通过调用`_get_next_sentence`函数从输入`paragraph`生成用于下一句预测的训练样本。这里`paragraph`是句子列表，其中每个句子都是词元列表。自变量`max_len`指定预训练期间的BERT输入序列的最大长度。

In [14]:
#@save
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    # 遍历段落中的句子（连续）
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑 1 个 '<cls>' 词元和 2 个 '<sep>' 词元；若长度超过限制，则跳过
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        # segments 为片段嵌入，0 表示第一句，1 表示第二句
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        # 返回值包含了 tokens， segments， is_next
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

### 生成遮蔽语言模型任务的数据

为了从BERT输入序列生成遮蔽语言模型的训练样本，我们定义了以下`_replace_mlm_tokens`函数。在其输入中，`tokens`是表示BERT输入序列的词元的列表，`candidate_pred_positions`是不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测），以及`num_mlm_preds`指示预测的数量（选择15%要预测的随机词元）。

在`subsec_mlm`中定义遮蔽语言模型任务之后，在每个预测位置，**输入可以由特殊的“掩码”词元或随机词元替换，或者保持不变。**

最后，该函数返回可能替换后的输入词元、发生预测的词元索引和这些预测的标签。

In [15]:
#@save
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    # 为遮蔽语言模型的输入创建新的词元副本，其中输入可能包含替换的 “<mask>” 或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取 15% 的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:

        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80% 的时间：将词替换为 “<mask>” 词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10% 的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10% 的时间：用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        
        # 使用 masked_token 替换 mlm_pred_position 位置的词元
        mlm_input_tokens[mlm_pred_position] = masked_token
        # 记录 被替换词元的位置 和 该位置的原始词元
        pred_positions_and_labels.append((mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

通过调用前述的`_replace_mlm_tokens`函数，以下函数将BERT输入序列（`tokens`）作为输入，并返回输入词元的索引（在`subsec_mlm`中描述的可能的词元替换之后）、发生预测的词元索引以及这些预测的标签索引。

In [16]:
#@save
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        # 候选预测位置
        candidate_pred_positions.append(i)

    # 遮蔽语言模型任务中预测 15% 的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    # 生成 MLM 输入，以及 被替换词元的位置 和 该位置的原始词元
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab)
    # 按照 被替换词元的位置 进行排序
    pred_positions_and_labels = sorted(pred_positions_and_labels, key=lambda x: x[0])
    # 掩蔽位置索引
    pred_positions = [v[0] for v in pred_positions_and_labels]
    # 对应原始词元，用作标签
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

## 将文本转换为预训练数据集

现在我们几乎准备好为BERT预训练定制一个`Dataset`类。在此之前，我们仍然需要定义辅助函数`_pad_bert_inputs`来将特殊的“&lt;mask&gt;”词元附加到输入。

它的参数`examples`包含来自两个预训练任务的辅助函数`_get_nsp_data_from_paragraph`和`_get_mlm_data_from_tokens`的输出。

这段代码中的 `_pad_bert_inputs` 函数用于将 BERT 模型训练的输入数据进行填充和格式化，**以适应固定的 `max_len` 长度**。具体来说，它将各输入示例扩展到相同长度，并生成掩蔽语言模型 (MLM) 和下一句预测 (NSP) 任务所需的标签和权重。

In [17]:
# 将各输入示例填充到相同长度，并生成掩蔽语言模型 (MLM) 和下一句预测 (NSP) 任务所需的标签和权重
#@save
def _pad_bert_inputs(examples, max_len, vocab):
    # 代表 MLM 任务中最大掩蔽数量（占总长度的 15%）
    max_num_mlm_preds = round(max_len * 0.15)
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    all_token_ids, all_segments, valid_lens,  = [], [], []
    nsp_labels = []

    for (token_ids, pred_positions, mlm_pred_label_ids, segments, is_next) in examples:
        # token_ids 扩展到 max_len 长度，不足的部分用 <pad> 词元 ID 填充
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (max_len - len(token_ids)), dtype=torch.long))
        # segments 扩展到 max_len 长度，不足的部分用 0 填充
        all_segments.append(torch.tensor(segments + [0] * (max_len - len(segments)), dtype=torch.long))
        # valid_lens 不包括 '<pad>' 的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        # nsp_labels 存储每个样本的下一句预测标签，表示当前句对是否为相邻句对
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))

        # pred_positions 填充到 max_num_mlm_preds 长度，不足的部分填充 0
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # all_mlm_weights 用于 MLM 任务中的损失计算。掩蔽位置权重为 1.0，填充部分权重为 0.0
        all_mlm_weights.append(torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.float32))
        # all_mlm_labels 包含掩蔽位置的标签，不足部分填充 0
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))

    return (all_token_ids, all_segments, valid_lens, all_pred_positions, all_mlm_weights, all_mlm_labels, nsp_labels)

将用于生成两个预训练任务的训练样本的辅助函数和用于填充输入的辅助函数放在一起，我们定义以下`_WikiTextDataset`类为用于预训练BERT的WikiText-2数据集。通过实现`__getitem__ `函数，我们可以任意访问WikiText-2语料库的一对句子生成的预训练样本（遮蔽语言模型和下一句预测）样本。

最初的BERT模型使用词表大小为30000的WordPiece嵌入`Wu.Schuster.Chen.ea.2016`。WordPiece的词元化方法是对`subsec_Byte_Pair_Encoding`中原有的字节对编码算法稍作修改。为简单起见，我们使用`d2l.tokenize`函数进行词元化。出现次数少于5次的不频繁词元将被过滤掉。

In [18]:
#@save
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入 paragraphs[i] 是代表段落的句子字符串列表
        # 而输出 paragraphs[i] 是代表段落的句子列表，其中每个句子都是词元列表
        paragraphs = [d2l.tokenize(paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs for sentence in paragraph]
        # 构建词表
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=['<pad>', '<mask>', '<cls>', '<sep>'])

        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(paragraph, paragraphs, self.vocab, max_len))  # extend 和 append 的区别
            
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab) + (segments, is_next))
                     for tokens, segments, is_next in examples]  # 遍历 NSP 数据的 examples，生成 MLM 数据
        
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(examples, max_len, self.vocab)

    # 用于按索引 idx 返回相应的数据样本，包括 NSP 和 MLM 任务的输入和标签
    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

通过使用`_read_wiki`函数和`_WikiTextDataset`类，我们定义了下面的`load_data_wiki`来下载并生成WikiText-2数据集，并从中生成预训练样本。

In [19]:
'''
#@save
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab
    '''

In [35]:
# 因原网页链接失效，使用 load_data_wiki1 替换原函数 load_data_wiki
def load_data_wiki1(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers()
    paragraphs = _read_wiki1('../data/train-00000-of-00001.parquet')
    train_set = _WikiTextDataset(paragraphs, max_len)  #返回数据集
    train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=0)
    return train_iter, train_set.vocab

将批量大小设置为 512，将 BERT 输入序列的最大长度设置为 64，我们打印出小批量的 BERT 预训练样本的形状。

注意，在每个 BERT 输入序列中，为遮蔽语言模型任务预测 $10$（$64 \times 0.15$）个位置。

下面代码的输出值：

- `tokens_X`：批次中的序列（即经过 BERT 编码的词元 ID）。 `(batch_size, max_len)`

- `segments_X`：段索引，表示输入是否来自同一段（句子A或B）。 `(batch_size, max_len)`

- `valid_lens_x`：有效长度，表示每个序列中不含 <pad> 的真实词元数量。 `(batch_size, )`

- `pred_positions_X`：在 MLM 任务中用于预测的词元位置（数量为 max_len * 0.15）。 `(batch_size, max_len * 0.15)`

- `mlm_weights_X`：掩码语言模型的权重，用于过滤填充词元的预测。 `(batch_size, max_len * 0.15)`

- `mlm_Y`：遮蔽语言模型的标签，用于计算损失。 `(batch_size, max_len * 0.15)`

- `nsp_y`：下一句预测（NSP）任务的标签，表示给定的两个句子是否是连续的。 `(batch_size, )`

In [36]:
batch_size, max_len = 512, 64
# 因原网页链接失效，使用用 load_data_wiki1 替换原函数 load_data_wiki
train_iter, vocab = load_data_wiki1(batch_size, max_len)

# 查看 train_iter 第一条数据信息
for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X, mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,nsp_y.shape)
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


最后，我们来看一下词量。即使在过滤掉不频繁的词元之后，它仍然比PTB数据集的大两倍以上。

In [34]:
len(vocab)

20256

## 小结

* 与PTB数据集相比，WikiText-2数据集保留了原来的标点符号、大小写和数字，并且比PTB数据集大了两倍多。
* 我们可以任意访问从WikiText-2语料库中的一对句子生成的预训练（遮蔽语言模型和下一句预测）样本。

## 练习

1. 为简单起见，句号用作拆分句子的唯一分隔符。尝试其他的句子拆分技术，比如Spacy和NLTK。以NLTK为例，需要先安装NLTK：`pip install nltk`。在代码中先`import nltk`。然后下载Punkt语句词元分析器：`nltk.download('punkt')`。要拆分句子，比如`sentences = 'This is great ! Why not ?'`，调用`nltk.tokenize.sent_tokenize(sentences)`将返回两个句子字符串的列表：`['This is great !', 'Why not ?']`。
2. 如果我们不过滤出一些不常见的词元，词量会有多大？

[Discussions](https://discuss.d2l.ai/t/5738)
